# Descubre tu rendimiento académico del siguiente semestre! 💯

### ¿Te has preguntado cómo será tu rendimiento en un semestre muy cargado? Te tenemos la solución!!! Podemos predecir con mediana certeza si es que bajarás, mantendrás o subirás tu rendimiento académico 👁

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import warnings
import random
from ipywidgets import (IntSlider, FloatSlider, HBox, VBox, Layout, 
Button, Text, BoundedFloatText, Label, IntProgress)

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data_2015 = pd.read_csv('datasets/2015_preprocesado.csv')
data_2016 = pd.read_csv('datasets/2016_preprocesado.csv')
data_2017 = pd.read_csv('datasets/2017_preprocesado.csv')

In [4]:
data = data_2015.append(data_2016.append(data_2017))

In [5]:
def first(i):
    return i.iloc[0]

In [6]:
nota_final = data.groupby(by=['COD ALUMNO', 'AÑO', 'SEMESTRE'])['NOTA FINAL'].apply(list).to_frame()
ramos = data.groupby(by=['COD ALUMNO', 'AÑO', 'SEMESTRE'])['SIGLA'].apply(list).to_frame()
ppa = data.groupby(by=['COD ALUMNO', 'AÑO', 
                            'SEMESTRE'])['PPA', 'CRÈDITOS_PREVIOS', 'MAX_PROMEDIO', 
                                         'MIN_PROMEDIO', 'MAX_SIGLA', 'MIN_SIGLA',
                                         'MAJOR CÓDIGO SELECCIONADO', 'MAJOR SELECCIONADO'].apply(first)
creditos_semestre = data.groupby(by=['COD ALUMNO', 'AÑO', 'SEMESTRE'])['CRÉDITOS CURSO'].apply(sum).to_frame()
creditos_semestre.rename(columns={'CRÉDITOS CURSO': 'CRÉDITOS SEMESTRE'}, inplace=True)

In [7]:
data_knn = pd.concat([ramos, nota_final, ppa, creditos_semestre], axis=1)
data_knn = data_knn.loc[data_knn['PPA'] != 0]

In [8]:
data_knn = data_knn.loc[data_knn['CRÈDITOS_PREVIOS'] <= 300]

In [9]:
X = data_knn[[col for col in data_knn.columns if col != 'NOTA FINAL']]
y = data_knn[['NOTA FINAL']]

In [10]:
def filtro(X, Y, row_X):
    ramos = row_X['SIGLA']
    ramos_filtrados = {}
    for index, ramo in enumerate(ramos):
        if ramo[0] == "I" or ramo[:3] in ["MAT", "BIO", "FIS", "QIM", "EYP"]:
            ramos_filtrados[ramo] = 0
        else:
            return False, False, False
            
    creditos_semestre = row_X['CRÉDITOS SEMESTRE']
    X2 = X.loc[(creditos_semestre + 5 >= X['CRÉDITOS SEMESTRE']) & (X['CRÉDITOS SEMESTRE'] >= creditos_semestre - 5)]
    Y = Y.loc[(creditos_semestre + 5 >= X['CRÉDITOS SEMESTRE']) & (X['CRÉDITOS SEMESTRE'] >= creditos_semestre - 5)]
    
    size = 0

    for ramo1 in ramos_filtrados:
        for ramo2 in ramos_filtrados:
            if ramo1 != ramo2:
                filtro_ramos = X2['SIGLA'].apply(lambda x: (ramo1 in x) and (ramo2 in x))
                new_X = X2[filtro_ramos]
                new_y = Y[filtro_ramos]
                if len(new_X) > size:
                    size = len(new_X)
                    maximo_X = new_X.copy()
                    maximo_y = new_y.copy()
                    ramos = [ramo1, ramo2]
                    
    if size == 0:
        return False, False, False
    
    for index, row in maximo_X.iterrows():
        
        siglas = row['SIGLA']
        y = maximo_y.loc[index]
        notas = y['NOTA FINAL']
        
        posiciones = []
        for ramo in ramos:
            posiciones.append(siglas.index(ramo))
        notas_ramos = []
        for posicion in posiciones:
            notas_ramos.append(notas[posicion])
        
        maximo_X.loc[index, 'RAMO 1'] = ramos[0]
        maximo_X.loc[index, 'RAMO 2'] = ramos[1]
        
        maximo_y.loc[index, 'NOTA RAMO 1'] = notas_ramos[0]
        maximo_y.loc[index, 'NOTA RAMO 2'] = notas_ramos[1]
        
    row_X['RAMO 1'] = ramos[0]
    row_X['RAMO 2'] = ramos[1]

    return maximo_X, maximo_y, row_X

In [11]:
# PPA hasta el momento 1-7
# crd previos 30 - 300, step 5
# 2 cursos que va a tomar ahora juntos
# mjr promedio, peor promedio 1-7
# crd que va a tomar ewe 20-65, step 5


In [12]:
layout = Layout(width='40vw', height='4vh') 

ppa = FloatSlider(description='PPA', min=1, max=7, step=.01, style={'description_width': 'initial'}, 
                  layout=layout)
crd_previos = IntSlider(description='Créditos totales hasta el semestre anterior', min=30, max=300, 
                        step=5, style={'description_width': 'initial'}, layout=layout)
mjr_promedio = FloatSlider(description='Mejor promedio hasta el momento', min=1, max=7, step=.01,
                         style={'description_width': 'initial'}, layout=layout)
peor_promedio = FloatSlider(description='Peor promedio hasta el momento', min=1, max=7, step=.01, 
                            style={'description_width': 'initial'}, layout=layout)
crd_inscritos = IntSlider(description='Créditos que se quieren inscribir', min=20, max=65, step=5, 
                            style={'description_width': 'initial'}, layout=layout)

sigla_1 = Text(
    value='',
    placeholder='Sigla',
    description='Sigla primer curso',
    disabled=False,
    style={'description_width': 'initial'}
)

sigla_2 = Text(
    value='',
    placeholder='Sigla',
    description='Sigla segundo curso',
    disabled=False,
    style={'description_width': 'initial'}
)

# mjr_promedio = BoundedFloatText(
#     value='',
#     min=1,
#     max=7.0,
#     step=0.01,
#     description='Mejor promedio hasta el momento',
#     disabled=False
# )

# slider_layout1 = HBox([ppa, crd_inscritos])
# slider_layout2 = HBox([mjr_promedio, peor_promedio])
# slider_layout3 = HBox([crd_previos])
# slider_layout4 = HBox([sigla_1, sigla_2])
button = Button(description="Predecir!", 
                button_style='success', 
                disabled=False)

label = Label(value='')

In [13]:
def predecir(modelo_knn, X_test):
    x1, y1, x2 = filtro(X, y, X_test)
    
    if type(x1) == bool:
        return 'Ingresa ramos con siglas de la malla del plan común o del departamento de Ingeniería'

    x2 = pd.DataFrame(x2).T
    
    x1 = x1.drop(['SIGLA', 'CRÉDITOS SEMESTRE', 'RAMO 1', 'RAMO 2'], axis=1)
    x2 = x2.drop(['SIGLA', 'CRÉDITOS SEMESTRE', 'RAMO 1', 'RAMO 2'], axis=1)
    y1 = y1.drop(['NOTA FINAL'], axis=1)


    x1_a = x1.drop(['MAJOR SELECCIONADO'], axis=1)
    
    x1_a = x1_a.drop(['MAJOR CÓDIGO SELECCIONADO', 'MIN_SIGLA', 'MAX_SIGLA'], axis=1)
    
    x2_a = x2
    try:
        x2_a = (x2_a - x1_a.min())/(x1_a.max() - x1_a.min())
        x1_a = (x1_a - x1_a.min())/(x1_a.max() - x1_a.min())
        
    except ZeroDivisionError:
        return 'No se pudo realizar la predicción'

    x1_a = x1_a.fillna(0)
    x2_a = x2_a.replace([np.inf, -np.inf], 0)
    if len(x1_a) < 5:
        return 'No se pudo realizar la predicción'

    
    y1_a = pd.DataFrame()
    y1_a['NOTA RAMO 1'] = pd.cut(y1['NOTA RAMO 1'], bins=[1, 2, 3, 4, 5, 6, 7])
    y1_a['NOTA RAMO 2'] = pd.cut(y1['NOTA RAMO 2'], bins=[1, 2, 3, 4, 5, 6, 7])
    y1_a['NOTA RAMO 1'] = y1_a['NOTA RAMO 1'].astype(str)
    y1_a['NOTA RAMO 2'] = y1_a['NOTA RAMO 2'].astype(str)


    modelo_knn.fit(x1_a, y1_a)
    pred = modelo_knn.predict(x2_a)

    y_predict = [j.replace('.0', '') for j in list(pred[0])]

    return y_predict


In [14]:
def my_dist(x,y):
    weight = np.ones((4,))
    weight[[1]] = 2
    weight = weight/weight.sum()
    dist = np.sqrt((x-y)**2)
    return np.dot(dist, weight)

In [15]:
def on_click(change):
    new_ppa = ppa.value
    new_crd_previos = crd_previos.value
    new_mjr_promedio = mjr_promedio.value
    new_peor_promedio = peor_promedio.value
    new_crd_inscritos = crd_inscritos.value
    new_sigla_1 = sigla_1.value
    new_sigla_2 = sigla_2.value
    notas_valores = {'(1, 2]': 1.5, '(2, 4]': 2.5, '(3, 4]': 3.5 ,'(4, 5]': 4.5, '(5, 6]': 5.5, '(6, 7]': 6.5}
    
    lista = [new_sigla_1.upper(), new_sigla_2.upper()]

    if new_sigla_1 != '' and new_sigla_2 != '':
        label.value = 'llegó aquí ewe'
        df = pd.DataFrame(np.array([[new_ppa, new_crd_previos, new_mjr_promedio, 
                                    new_peor_promedio, new_crd_inscritos, lista]]), 
                          columns=['PPA', 'CRÈDITOS_PREVIOS', 'MAX_PROMEDIO', 'MIN_PROMEDIO',
                                  'CRÉDITOS SEMESTRE', 'SIGLA'])
        
        # progress.value += 1
        knn = KNeighborsClassifier(n_neighbors=5, metric=my_dist)
        prediccion = predecir(knn, df.iloc[0])
        label.value = 'procesando!!!'
        if 'ramos' in prediccion or 'No' in prediccion:
            label.value = prediccion
        else:
            resultado = f'El promedio de tu primer curso estará entre {prediccion[0][1]} y {prediccion[0][4]}, y el de tu segundo curso estará entre {prediccion[1][1]} y {prediccion[1][4]}. '
            promedio = (notas_valores[prediccion[0]] + notas_valores[prediccion[1]])/2
            if promedio > new_ppa:
                resultado += '\nTu PPA puede subir si rindes estos cursos juntos 🎉. Tu rendimiento puede subir ✨'
            elif promedio < new_ppa:
                resultado += '\nTu PPA puede bajar si rindes estos cursos juntos 🥺. Tu rendimiento puede bajar, ten cuidado.'
            else:
                resultado += '\nTu PPA no cambiará si rindes estos cursos juntos 🧐. Tu rendimiento se mantendrá 🤝'
            label.value = resultado
        
    else:
        prediccion = 'Debes ingresar 2 cursos!'
        label.value = prediccion
    

In [16]:
button.on_click(on_click)

In [17]:
final_layout = VBox([ppa, crd_inscritos, mjr_promedio, peor_promedio, crd_previos, 
                     sigla_1, sigla_2, button, label])
final_layout